In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 2

@author: yaning
"""

import numpy as np
import importlib
import matplotlib.pyplot as plt
import torch
from torch.distributions import Normal, Uniform
from tqdm import tqdm

# functions and classes that i wrote
import with_learning.run_network as run_network
import with_learning.learning_NN.Receptors as Receptors
import with_learning.learning_NN.Network as Network

importlib.reload(run_network)
importlib.reload(Receptors)
importlib.reload(Network)

np.set_printoptions(threshold=np.inf)

In [15]:
# functions
# get the minimum and maximum of the voltages
def get_min_max_distance(voltages):
    min_voltage = abs(np.nanmin(voltages) + 70)
    max_voltage = abs(np.nanmax(voltages) - 40.1)
    return min_voltage, max_voltage

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def get_current_distance(currents):
    current_d = abs(np.nanmax(currents)-175)
    return current_d


In [16]:
# # try use MCMC result params
# path = "/home/yaning/Documents/Spiking_NN/without_learning/"
# samples = np.load(path + "MCMC_samples/static_std_initial_0.npy")
# cut_samples = samples[1000:, :]
# values = np.mean(cut_samples, axis=0)
# infer_names = Receptors.LigandGatedChannelFactory.infer_names
# infer_params = dict(zip(infer_names, values))

infer_names = Receptors.LigandGatedChannelFactory.infer_names
infer_params = Receptors.LigandGatedChannelFactory.infer_params


# set only the between 0-1 the initial parameters to 0 
# because the normal distribution's mean should at 0 
# so the initial value after sigmoid function will be 0.5 (in the middle)
for i, key in enumerate(infer_params):
    if i <= 9:
        infer_params[key] = 0.5
    else:
        infer_params[key] = 20

# run first round
# return voltages, currents, neuron_names
voltages, currents, neuron_names = run_network.run(infer_params)

min_voltage_old, max_voltage_old = get_min_max_distance(voltages)
current_d_old = get_current_distance(currents)


samples = []
# pure_samples = []
num = 15
for i in tqdm(range(num), desc="Processing", ncols=100):
    # sampling
    # std = np.exp(-i*3/num)
    std = 1
    # one_round_pure_sample = []
    one_round_sample = []
    
    temp_infer_params = infer_params
    
    for j in range(len(infer_names)):
        if j <= 9:
            temp_sample = Normal(infer_params[infer_names[j]], std).sample()
            # one_round_pure_sample.append(temp_sample)
            # temp_infer_params[infer_names[j]] = sigmoid(temp_sample)
            temp_infer_params[infer_names[j]] = sigmoid(temp_sample)
        else:
            # i decided to not set a range for the ones that can go beyond 0-1
            # because if you set a range, the sigmoid very extremly large from the first sample
            # (see the MCMC_run.py in without learning for setting range)
            temp_sample = Normal(infer_params[infer_names[j]], std).sample()
            # one_round_pure_sample.append(temp_sample)
            temp_infer_params[infer_names[j]] = temp_sample
    
    for j in infer_names:
        temp_sample = Normal(infer_params[j], std).sample()
    


    # run with sampled value
    voltages, currents, neuron_names = run_network.run(temp_infer_params)
    
    min_voltage_new, max_voltage_new = get_min_max_distance(voltages)
    current_d_new = get_current_distance(currents)
    
    # get the rates 
    # min_voltage_rate = min_voltage_new/min_voltage_old
    # max_voltage_rate = max_voltage_new/max_voltage_old
    # current_rate = current_d_new/current_d_old
    
    old_scores = np.array([min_voltage_old, max_voltage_old, current_d_old])
    new_scores = np.array([min_voltage_new, max_voltage_new, current_d_new])

    lambda_value = 0.01
    old_softmax = np.exp(lambda_value * old_scores) / np.sum(np.exp(lambda_value * old_scores))
    new_softmax = np.exp(lambda_value * new_scores) / np.sum(np.exp(lambda_value * new_scores))
    
    acceptance_ratio = np.sum(old_softmax) / np.sum(new_softmax)
    
    u = np.random.uniform(0, 1)


    if acceptance_ratio >= u:
        min_voltage_old = min_voltage_new
        max_voltage_old = max_voltage_new
        current_d_old = current_d_new
        
        infer_params = temp_infer_params

    
    for name in infer_names:
        one_round_sample.append(infer_params[name])
    
    samples.append(one_round_sample)
    pure_samples.append(one_round_pure_sample)
    

samples = np.array(samples)
pure_samples = np.array(pure_samples)


np.save(path + 'samples.npy', samples)
np.save(path + 'pure_samples.npy', pure_samples)

/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:79: RuntimeWarning: overflow encountered in exp
  self.m.alpha = .1*((25-self.Vm) / (np.exp((25-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:80: RuntimeWarning: overflow encountered in exp
  self.m.beta = 4*np.exp(-self.Vm/18)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:81: RuntimeWarning: overflow encountered in exp
  self.n.alpha = .01 * ((10-self.Vm) / (np.exp((10-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:82: RuntimeWarning: overflow encountered in exp
  self.n.beta = .125*np.exp(-self.Vm/80)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:83: RuntimeWarning: overflow encountered in exp
  self.h.alpha = .07*np.exp(-self.Vm/20)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:84: RuntimeWarning: overflow encountered in exp
  self.h.beta = 1/(np